In [ ]:
# !pip install tensorflow

In [ ]:
# !pip uninstall ray
# !pip install -U "<PREFIX PATH>\ray-2.0.0.dev0-cp38-cp38-win_amd64.whl"

In [1567]:
# !python --version

Python 3.8.8


In [1]:
import numpy as np
import random
from tensorflow import keras
import ray
np.seterr(all="ignore")


{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [3]:
ray.init(num_cpus=2)

RayContext(dashboard_url='', python_version='3.8.8', ray_version='2.0.0.dev0', ray_commit='a34dcfce85deaaa4590b7edd68649af16ac8b571', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': 'tcp://127.0.0.1:62214', 'raylet_socket_name': 'tcp://127.0.0.1:65174', 'webui_url': '', 'session_dir': 'C:\\Users\\Roman\\AppData\\Local\\Temp\\ray\\session_2022-04-23_14-05-35_860726_60840', 'metrics_export_port': 63505, 'gcs_address': '127.0.0.1:63884', 'address': '127.0.0.1:63884', 'node_id': '194daac6e14c1e3f4c94f796ae015e1316eb5670290016685585428c'})

In [16]:
ray.shutdown()

In [30]:
np.array((4,))

array([4])

In [13]:

test_model = Model(
    (
        layer(3, None, False),
        layer(2, None, False),
        layer(1, None, False)
    ),
    mse_loss_set
)

test_model.weights = [
    np.array([
        [0.1, 0.2],
        [0.3, 0.4],
        [0.5, 0.6]
    ]),
    np.array([
        [0.3],
        [0.6]
    ])
]

# test_model.log_pool_info()

# expected result is 2.34
test_model.forward([1, 2, 3])

array([2.34])

[0 0] [0.73714542]
[0 1] [0.759092]
[1 0] [0.80487922]
[1 1] [0.82386688]

epoch: 0
[0 0] [0.73714542]
[0 1] [0.759092]
[1 0] [0.80487922]
[1 1] [0.82386688]
--------------------
epoch: 400
[0 0] [-0.0993933]
[0 1] [0.66363996]
[1 0] [0.66363091]
[1 1] [0.3613894]
--------------------
epoch: 800
[0 0] [-0.12663424]
[0 1] [0.71768317]
[1 0] [0.71768227]
[1 1] [0.27750187]
--------------------
epoch: 1200
[0 0] [-0.14083222]
[0 1] [0.75160192]
[1 0] [0.75160182]
[1 1] [0.1990651]
--------------------
epoch: 1600
[0 0] [-0.14968919]
[0 1] [0.77705273]
[1 0] [0.77705272]
[1 1] [0.13386574]
--------------------
epoch: 2000
[0 0] [-0.15553041]
[0 1] [0.79656388]
[1 0] [0.79656388]
[1 1] [0.08188127]
--------------------
epoch: 2400
[0 0] [-0.15949746]
[0 1] [0.81149917]
[1 0] [0.81149917]
[1 1] [0.04135619]
--------------------
epoch: 2800
[0 0] [-0.16224065]
[0 1] [0.82286603]
[1 0] [0.82286603]
[1 1] [0.01026989]
--------------------
epoch: 3200
[0 0] [-0.16415667]
[0 1] [0.83146467]
[1 0]

In [24]:
import torchvision
import torch

transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])
torch_ds = torchvision.datasets.MNIST('/files/', train=True, download=True)

In [14]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [29]:
# torch_ds[54][0]

In [14]:
def flatten(t):
    return [item for sublist in t for item in sublist]

train_dataset = []

for i in range(len(torch_ds)):
    img, tar = torch_ds[i]
    inp = transform(img).numpy().flatten()
    target = [0] * 10
    target[tar] = 1
    train_dataset.append((inp, target, img))
    


In [15]:
layout = (
    layer(784, None),
    layer(64, sigmoid_activator),
    layer(10, sigmoid_activator),
)

model = Model(layout, mse_loss_set)

gradient_calculator = RayGradientCalculator(model, 64)
rms_prop_optimizer = RMSPropOptimizer(model, 0.01, 0.01, 0.01)


In [16]:
BATCH_SIZE = 64
batches = []

next_batch = []

for inp, tar, img in train_dataset:
    next_batch.append((inp, tar))
    if len(next_batch) == BATCH_SIZE:
        batches.append(next_batch)
        next_batch = []

In [17]:
def test(model, train_dataset, count=512):
    guessed = 0
    for _ in range(count):
        inp, tar, _ = random.choice(train_dataset)
        res = model.forward(inp)
        if tar[list(res).index(max(res))] == 1:
            guessed += 1
    return guessed / count

In [18]:
model.weights_layout()

[(785, 65), (65, 10)]

In [19]:
from IPython.display import clear_output
import time

err = []
guessed = 0

testing_step = 100

for epoch in range(50):
    print('@@@@@@@@@@@@@@@@@@@@@')
    print(f'EPOCH: {epoch+1}')
    print('@@@@@@@@@@@@@@@@@@@@@')

    last_test_time = time.time()
    for i, batch in enumerate(batches):
        grad = gradient_calculator.run(batch)
        # print(list(map(lambda x: x.shape, grad)))
        rms_prop_optimizer.step(grad)

        if i % testing_step == testing_step - 1:
            now = time.time()
            elapsed = now - last_test_time
            last_test_time = now
            
            time_per_epoch = elapsed * len(batches) / testing_step
            
            print(f'Testing: accuracy={test(model, train_dataset)}')
            print(f'Time since last testing (seconds) {elapsed}')
            print(f'Estimated time per epoch:')
            print(f'    seconds = {time_per_epoch}')
            print(f'    minutes = {time_per_epoch / 60}')
            print(f'    hours   = {time_per_epoch / 60 / 60}')
            print(f'----------------------------------------')


@@@@@@@@@@@@@@@@@@@@@
EPOCH: 1
@@@@@@@@@@@@@@@@@@@@@


2022-04-23 04:11:15,076	WARNING worker.py:1398 -- WARNING: 48 PYTHON worker processes have been started on node: 5c46cbfedfe6d952f4786ccede9ea3259d6b0ea75446959eb9a6b0b9 with address: 127.0.0.1. This could be a result of using a large number of actors, or due to tasks blocked in ray.get() calls (see https://github.com/ray-project/ray/issues/3644 for some discussion of workarounds).
2022-04-23 04:11:17,147	WARNING worker.py:1398 -- WARNING: 61 PYTHON worker processes have been started on node: 5c46cbfedfe6d952f4786ccede9ea3259d6b0ea75446959eb9a6b0b9 with address: 127.0.0.1. This could be a result of using a large number of actors, or due to tasks blocked in ray.get() calls (see https://github.com/ray-project/ray/issues/3644 for some discussion of workarounds).


Testing: accuracy=0.1484375
Time since last testing (seconds) 20.91240096092224
Estimated time per epoch:
    seconds = 195.9491970038414
    minutes = 3.265819950064023
    hours   = 0.054430332501067054
----------------------------------------
Testing: accuracy=0.33984375
Time since last testing (seconds) 13.093400239944458
Estimated time per epoch:
    seconds = 122.68516024827957
    minutes = 2.0447526708046597
    hours   = 0.03407921118007766
----------------------------------------
Testing: accuracy=0.509765625
Time since last testing (seconds) 15.629865646362305
Estimated time per epoch:
    seconds = 146.4518411064148
    minutes = 2.4408640184402466
    hours   = 0.04068106697400411
----------------------------------------


KeyboardInterrupt: 

In [297]:
idx = random.randint(0, len(train_dataset))
inp, _, img = train_dataset[idx]

res = model.forward(inp)
pred = list(res).index(res.max())
print(pred)
img

6


In [286]:

print(train_dataset[0][0])

[-0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0

In [ ]:
import time

time.time()

In [ ]:
softmax_d(np.array(39))

In [ ]:
IND = random.randint(1, 10000)
inp = train_dataset[IND][0]
tar = train_dataset[IND][1]

res = model.forward(inp)
print(res)
print(tar)
print(sum(abs(res - tar)))